<p style="text-align: center; font-size: 300%"> Computational Finance </p>
<img src="img/ABSlogo.svg" alt="LOGO" style="display:block; margin-left: auto; margin-right: auto; width: 50%;">

# Monte Carlo Methods
## Brownian Motion
* We saw last week that the binomial tree implies for $X_t\equiv \log S_t$ that
$$
X_{i\delta t}=X_{(i-1)\delta t}+R_i \Longleftrightarrow
\Delta X_i = R_i,\tag{$\dagger$}
$$
  where $R_i=\log u$ or $R_i=\log d$, with probabilities $\mathbb{Q}[u]$ and $\mathbb{Q}[d]$. 
* Equation ($\dagger$) is a *stochastic difference equation*.
* Its *solution* 
$$
X_T=\log S_0 +\sum_{i=1}^N R_i=\log S_0 +\sigma \sqrt{\delta t}(2k-N)
$$
is called a *binomial process*, or in the special case with
$\mathbb{E}[R_i]=0$, a *random walk*.


* We also saw that if we let $N\rightarrow\infty$ (so that $\delta t \rightarrow 0$),
$$
X_T-X_0\stackrel{d}{\rightarrow}N(\mu T,\sigma^2 T),\quad \mu\equiv r-\frac{1}{2}\sigma^2.
$$
* The argument can be repeated for every $X_t, t\leq T$, showing that
$$
X_t-X_0\stackrel{d}{\rightarrow}N(\mu t,\sigma^2 t),
$$
and that for any $0<t<T$, $X_t-X_0$ and $X_T-X_t$ are independent.
* As $\delta t\rightarrow 0$, $\{X_t\}_{t\geq 0}$ becomes a continuous time process: the indexing set is now given by the entire positive real line.
* This continuous time limit (with $\mu=0$ and $\sigma^2=1$) is called *Brownian motion*, or *Wiener process*.
* From now on, rather than modelling in discrete time and then letting $\delta t \rightarrow 0$, we will directly model in continuous time, using Brownian motion as a building block.


* Definition of (standard) *Brownian Motion*:
Stochastic process $\{W_{t}\}_{t\geq 0}$ satisfying
  * $W_{0}=0$;
  * The increments $W_{t} - W_s$ are independent for all $0 \le s < t$;
  * $W_{t} - W_s \sim N(0,t-s)$ for all $0 \le s \le t$;
  * Continuous sample paths.

* This is standard Brownian motion, whereas $X_{t}=\sigma W_{t}$ is Brownian motion with variance $\sigma ^{2}$.
* Restriction that process start at zero may be loosened by considering $X_{t}=X_{0}+\sigma W_{t}$.
* Brownian motion with drift: $X_{t}=X_{0}+\mu t+\sigma W_{t}$, so that $\mathbb{E}[X_t]=X_{0}+\mu t$, $\mathrm{Var}[X_t]=\sigma^{2}t$.


* Properties of Brownian Sample Paths:
  * *Continuity*: by assumption, and also $W_{t+\delta t}-W_{t}\sim N(0,\delta t)\rightarrow 0$ as $\delta t\downarrow0$;
  * *Nowhere differentiability*: intuitively, this is seen from
$$
\frac{W_{t}-W_{t-\delta t}}{\delta t}\sim N\left( 0,\frac{1}{\delta t}\right) ,\quad \frac{%
W_{t+\delta t}-W_{t}}{\delta t}\sim N\left( 0,\frac{1}{\delta t}\right) ;
$$
left and right difference quotients do not have (common) limit as $\delta t\downarrow 0$.
  * *Self-similarity*: Zooming in on a Brownian motion yields another Brownian motion: for any $c>0$, $X_{t}=\sqrt{c}W_{t/c}$ is
a Brownian motion.



In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
%matplotlib inline
np.random.seed(0)
def bmsim(T, N, X0=0, mu=0, sigma=1):
    """
    Simulate a Brownian motion path.
    """
    deltaT=float(T)/N
    tvec=np.linspace(0, T, N+1)
    z=np.random.randn(N)    
    dX=mu*deltaT+sigma*np.sqrt(deltaT)*z #X[j+1]-X[j]=mu*deltaT+sigma*np.sqrt(deltaT)*z[j]
    X=np.cumsum(dX)
    X=np.insert(X, 0, 0.) #X[0]=0
    return pd.Series(X, index=tvec)
  
W=bmsim(1, 1000)
W.plot()
plt.title('Simulated Brownian Motion Path')
plt.xlabel("$t$")
plt.ylabel("$W_t$")
plt.savefig("img/BMpath.svg")
plt.close()

<img src="img/BMpath.svg" alt="Simulated Brownian Motion Path" style="display:block; margin-left: auto; margin-right: auto; width: 80%;">

* Brownian motion itself is not a very useful model for stock prices, because it can become negative. Instead we model $X_t\equiv \log S_t$ as a Brownian motion with drift:

\begin{align*}
X_{t}&=X_{0}+\mu t+\sigma W_{t}\mbox{, so that}\\
S_t&=\exp(X_t)\\
&=S_0\exp(\mu t+\sigma W_{t}).
\end{align*}
* The resulting process for $S_t$ is called *Geometric Brownian motion* (GBM).
* This implies that the log return $\log S_t-\log S_s=X_t-X_s$, $s\lt t$,
is independent of $X_{s}$, with constant variance for fixed $(t-s)$.

In [2]:
X=np.exp(W)
X.plot()
plt.title('Simulated GBM Path, $\mu=0$, $\sigma=1$.')
plt.xlabel("$t$")
plt.ylabel("$\exp(W_t)$")
plt.savefig("img/GBMpath.svg")
plt.close()

<img src="img/GBMpath.svg" alt="Simulated GBM Path" style="display:block; margin-left: auto; margin-right: auto; width: 80%;">

## Continuous Time Martingales
* In continuous time, a process $\{X_{t}\}_{t\geq 0}$ is a *martingale* if
  * $\mathbb{E}[|X_{t}|]<\infty $, for all $t\ge 0$; 
  * $\mathbb{E}[X_{t}|\mathcal{F}_{s}]=X_{s}$, for all $t>s\ge 0$,
where $\mathcal{F}_{t}$ denotes the information on $X_t$ up to time $t$.

* E.g., for Brownian motion
  * $\mathbb{E}[|W_{t}|]<\infty $ because $W_{t}\sim N(0,t)$;
  * $\mathbb{E}[W_{t}|\mathcal{F}_{s}] =\mathbb{E}[W_{s}+(W_{t}-W_{s})|\mathcal{F}_{s}]=W_{s}+0$ because of independent increments.

* For Geometric Brownian motion, $X_{t}=X_{0}\exp \left( \mu t+\sigma W_{t}\right)$, so that $X_{t}=X_{s}\exp \left( \mu (t-s)+\sigma (W_{t}-W_s)\right)$. Thus

\begin{align*}
\mathbb{E}[X_{t}|\mathcal{F}_{s}] &=\mathbb{E}\left[ X_{s}\exp \left(
\mu (t-s)+\sigma (W_{t}-W_{s})\right) |\mathcal{F}_{s}\right] \\
&=X_{s}\exp \left( \mu (t-s)\right) \mathbb{E}\left[ \exp \left( \sigma
(W_{t}-W_{s})\right) \right] \\
&=X_{s}\exp \left( \mu (t-s)\right) \exp \left( \tfrac{1}{2}\sigma
^{2}(t-s)\right) .
\end{align*}
* The last line above follows because $\mathbb{E}[\exp(z)]=\mu+\frac{1}{2}\sigma^2$ if $z\sim\mathrm{N}(\mu, \sigma^2)$. The distribution of $\exp(z)$ is called the *lognormal*.
* Hence GBM is a martingale only if $\mu =-\tfrac{1}{2}\sigma ^{2}$.

## Ito Processes
* Ito processes generalize Brownian motion with drift by allowing the drift and volatility to be time-varying and potentially stochastic.
* The trick is to describe the dynamics of a process with a *stochastic differential equation* (SDE), the continuous time equivalent of a stochastic difference equation.
* Take, for example, Brownian motion with drift,
$
X_t=X_0+\mu t+\sigma W_t.
$
* We know from calculus that
$$
\int_\tau^t \mu ds=\mu\int_\tau^t  ds=\mu (t-\tau).
$$
* If we define $\int_{\tau}^t dW_s=W_t-W_\tau$, then we see that
$$
X_t=X_0+\int_{0}^t\mu ds + \int_{0}^t \sigma dW_s.
$$

* This is often written in differential form as
$$
dX_t=\mu d t + \sigma dW_t.
$$
Note that this is just short hand notation for the integral form.
* An Ito process generalizes this by allowing $\mu$ and $\sigma$ to be time-varying and stochastic:
$$
dX_t=\mu_t d t + \sigma_t dW_t.\tag{$\dagger$}
$$
* Again, this is just short-hand for 
$$
X_t=X_0 +\int_0^t \mu_s d s + \int_0^t \sigma_s dW_s,
$$
where we define
$$
\int_{0}^{T}\mu_sds\equiv\lim_{n\rightarrow \infty
}\sum_{i=0}^{N-1}\mu(t_{i})\Delta t_{i+1},
\quad
\int_{0}^{T}\sigma(t)dW_t\equiv\lim_{n\rightarrow \infty
}\sum_{i=0}^{N-1}\sigma(t_{i})\Delta W_{t_{i+1}},
$$
$t_i\equiv iT/N$, $\Delta t_{i+1}\equiv t_{i+1}-t_{i}$, and $\Delta W_{t_{i+1}}\equiv [W_{t_{i+1}}-W_{t_{i}}]$.

* Remarks:
  * $X_t$ is the sum of two integrals. The first is called a *Riemann integral*, the second is an *Ito integral*. 
  * **Do not** think of the integrals as an *area under the curve* like in high school. Your intuition for the Ito integral should be that we are summing infinitesimally small Brownian increments $dW_t$, each scaled by the instantaneous volatility $\sigma_t$.
  * If $\mu_t$ and $\sigma_t$ depends only on the current $W_t$, then ($\dagger$) is called a *stochastic differential equation*. Example: $\mu (t,x)=\mu x$ and $\sigma (t,x)=\sigma x$, so that
$$
dX_{t}=\mu X_{t}dt+\sigma X_{t}dW_{t}.
$$
  * The *solution* to an SDE is an equation that describes $X_t$ in terms of just $W_t$ (i.e., $X_t$ does not appear on the RHS). Often, Ito's lemma is helpful in finding it.

## Ito's Lemma
* Ito's lemma answers the question: if $X_t$ is an Ito process with given dynamics, then what are the dynamics of a function $f(t,X_t)$?
* It can be stated as follows: Let $\{X_{t}\}_{t\ge0}$ be an Ito process satisfying $dX_{t} = \mu_{t}dt
+ \sigma_{t}dW_{t}$, and consider a function $f : \mathbb{R}^{+} \times \mathbb{R} \to \mathbb{R}$
with continuous partial derivatives
$$
\dot{f}(t,x)=\frac{\partial f(t,x)}{\partial t},\qquad f^{\prime }(t,x)=\frac{\partial f(t,x)}{\partial x},\qquad f^{\prime \prime }(t,x)=\frac{\partial ^{2}f(t,x)}{\partial x^{2}}.
$$
Then
$$
df(t,X_{t})=\dot{f}(t,X_{t})dt+f^{\prime }(t,X_{t})dX_{t}+\tfrac{1}{2}
f^{\prime \prime }(t,X_{t})\sigma _{t}^{2}dt.
$$

* Example: Geometric Brownian Motion. Let
$dS_t=S_t \mu dt +S_t \sigma dW_t\tag{$\ddagger$}$
and take $X_t=f(S_t)=\log S_t$. Then $\dot f (S_t) = 0$, $ f' (S_t) = 1/S_t$, $f'' (S_t) = -1/S_t^2$, and

\begin{align*}
dX_t=d f(S_t)&= \dot f (S_t) dt +  f' (S_t)  dS_t + \frac{1}{2} f'' (S_t) (S_t \sigma)^2 dt\\
        &= \frac{1}{S_t}  dS_t - \frac{1}{2S_t^2} (S_t \sigma)^2  dt\\
        &= \frac{1}{S_t}  (S_t \mu dt +S_t \sigma dW_t) - \frac{1}{2} \sigma^2 dt\\
        &=  \nu dt + \sigma dW_t, \quad \nu=\mu-\frac{1}{2}\sigma^2
\end{align*}
* I.e., ($\ddagger$) is the SDE for GBM:  $S_t=\exp(X_t)=S_0\exp(\nu t + \sigma W_t)$.

* Intuition (see Hull, 2012, Appendix to Ch. 13): In standard calculus, the total differential
$$
df=\dot{f}(t,g(t))dt+f^{\prime }(t,g(t))dg(t)
$$
is the linear part of a Taylor expansion; the remaining terms
are of smaller order as $dt, dg(t) \to 0$, so the total differential is a local linear approximation to $f$.
* If $g(t)=X_t$, an Ito process, take a 2nd order Taylor approximation:

\begin{align*}
&\delta f\approx \dot{f}(t,X_{t})\delta t+f^{\prime
}(t,X_{t})\delta X_{t} \\
&\qquad \qquad \qquad \qquad +\frac{1}{2}\left[ \frac{\partial ^{2}f}{%
\partial t^{2}}(\delta t)^{2}+2\frac{\partial ^{2}f}{\partial t\partial X_{t}%
}(\delta t)(\delta X_{t})+\frac{\partial ^{2}f}{\partial X_{t}^{2}}(\delta
X_{t})^{2}\right].
\end{align*}

* We have that $\delta X_{t}=(X_{t+\delta t}-X_{t})\approx \mu_t \delta t + \sigma_t \delta W_{t} \sim N(\mu_t \delta t,\sigma^2_t \delta t)$. Thus, $\mathbb{E} [(\delta X_{t})^2]\approx (\mu_t\delta t)^2+\sigma^2_t\delta t\approx \sigma^2_t\delta t $; i.e., the 2nd order term is of the same order
of magnitude as the 1st order term $\delta t$. 
* It can be shown that as $\delta t\rightarrow 0$, $(\delta X_{t})^{2}$ can be treated as non-stochastic: $(dX_{t})^{2}=\sigma _{t}^{2}dt$.
Together with $(dt)^{2}=0$ and $(dt)(dX_{t})=0$ this gives the result.